# 01_04_make_fits_summary

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name==version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [3]:
#import sys
#%pip install photutils==1.6 ccdproc==2.4

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [4]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        #print(f"**** module {pkg} is not installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module photutils is installed
**** module ccdproc is installed
**** module version_information is installed
This notebook was generated at 2023-09-24 13:44:17 (KST = GMT+0900) 
0 Python     3.11.4 64bit [GCC 11.2.0]
1 IPython    8.12.2
2 OS         Linux 5.15.0 84 generic x86_64 with glibc2.31
3 numpy      1.25.2
4 pandas     2.0.3
5 matplotlib 3.7.2
6 scipy      1.11.1
7 astropy    5.2.1
8 photutils  1.6.0
9 ccdproc    2.4.0
10 version_information 1.0.4


### import modules

In [5]:
import os
from glob import glob
from pathlib import Path
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
from ccdproc import CCDData

from astropy.time import Time
from datetime import datetime, timedelta
import ysfitsutilpy as yfu
import ysphotutilpy as ypu

import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

In [12]:
#%%
#######################################################
# read all files in base directory for processing
BASEDIR = Path("/mnt/Rdata/OBS_data") 
DOINGDIR = Path('/mnt/Rdata/OBS_data/A3_CCD_obs_raw/')

CCDDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(DOINGDIR))
print ("CCDDIRs: ", CCDDIRs)
print ("len(CCDDIRs): ", len(CCDDIRs))

CCDDIRs:  ['/mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_3bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/ST-8300M_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/ST-8300M_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STF-8300M_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STF-8300M_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STF-8300M_3bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STL-11000M_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STL-11000M_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STX-16803_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/STX-16803_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/TT-2600CP_1bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/TT-2600CP_2bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/TT-2600CP_3bin/', '/mnt/Rdata/OBS_data/A3_CCD_obs_raw/TT-2600CP_4bin/']
len(CCDDIRs):  16


In [ ]:
#######################################################
checkTime_ccd = 1
checkTime_ccdoptic = -2
checkTime_eachdir = -2

In [13]:
#%%
#######################################################
checkTime = 0
for CCDDIR in CCDDIRs[:] :
    doing_status = False
    CCDDIR = Path(CCDDIR)
    ccd_fpath = Path(f"{CCDDIR.parents[1] / CCDDIR.parts[-1]}.csv")
    print("ccd_fpath", ccd_fpath)

ccd_fpath /mnt/Rdata/OBS_data/QSI683ws_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/QSI683ws_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/QSI683ws_3bin.csv
ccd_fpath /mnt/Rdata/OBS_data/ST-8300M_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/ST-8300M_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STF-8300M_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STF-8300M_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STF-8300M_3bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STL-11000M_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STL-11000M_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STX-16803_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/STX-16803_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/TT-2600CP_1bin.csv
ccd_fpath /mnt/Rdata/OBS_data/TT-2600CP_2bin.csv
ccd_fpath /mnt/Rdata/OBS_data/TT-2600CP_3bin.csv
ccd_fpath /mnt/Rdata/OBS_data/TT-2600CP_4bin.csv


In [8]:
# DOINGDIR = DOINGDIRs[1]
# DOINGDIR = Path(DOINGDIR)
# ccd_fpath = Path(f"{DOINGDIR/DOINGDIR.parts[-1]}.csv")
# print("ccd_fpath", ccd_fpath)

# CCDOPTICs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
# print ("CCDOPTICDIRs: ", CCDOPTICDIRs)
# print ("len(DOINGSUBDIRs): ", len(CCDOPTICDIRs))

# remove = 'Cal'
# CCDOPTICs = [x for x in CCDOPTICDIRs if remove not in x]
# print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
# print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

# if ccd_fpath.exists() :
#     print(f"{str(ccd_fpath)} is already exist and is not old...")
#     t = os.path.getmtime(ccd_fpath)
#     ccd_fpath_dt = datetime.fromtimestamp(t)
#     print("ccd_fpath_dt: ", ccd_fpath_dt)
#     if ccd_fpath_dt < datetime.now() + timedelta(weeks=-2) :
#         print(f"{str(ccd_fpath)} is not old...")
            
#     else : 
#         summary_all = pd.DataFrame()
#         print(summary_all)    

# else : 
#     summary_all = pd.DataFrame()
#     print(summary_all) 


In [9]:
# for DOINGSUBDIR in DOINGSUBDIRs[:1] :
#     DOINGSUBDIR = Path(DOINGSUBDIR)
#     print("DOINGSUBDIR", DOINGSUBDIR)
#     fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
#     print("fits_in_dir", fits_in_dir)
#     print("len(fits_in_dir)", len(fits_in_dir))
#     if len(fits_in_dir) == 0 :
#         print(f"There is no fits fils in {DOINGSUBDIR}")
#         pass
#     else : 
#         save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
#         save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
#         print (f"Starting...\n{DOINGSUBDIR.name}")
#         if save_fpath2.exists():
#             os.remove(str(save_fpath2))
#             print (f"{str(save_fpath2)} is deleted...")
        
#         t = os.path.getmtime(save_fpath)
#         save_fpath_dt = datetime.fromtimestamp(t)
#         #print("save_fpath_dt: ", save_fpath_dt)
#         if save_fpath.exists() \
#             and save_fpath_dt < datetime.now() + timedelta(weeks=-2) :
#             print(f"{str(save_fpath)} is already exist and is not old...")
#             summary = pd.read_csv(str(save_fpath))
        
#         else : 
#             summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
#                         output = save_fpath,
#                         verbose = False
#                         )
#             print(f"{save_fpath} is created...")
#         summary_all = pd.concat([summary_all, summary], axis = 0)

# summary_all.reset_index(inplace=True)
# summary_all.to_csv(str(ccd_fpath))
# print(f"{ccd_fpath} is created...")  

In [11]:
#%%
#######################################################
checkTime = -2
for DOINGDIR in DOINGDIRs[:] :
    doing_status = False
    DOINGDIR = Path(DOINGDIR)
    ccd_fpath = Path(f"{DOINGDIR/DOINGDIR.parts[-1]}.csv")
    print("ccd_fpath", ccd_fpath)

    summary_ccd = pd.DataFrame()
    print(summary_ccd)   

    if not ccd_fpath.exists() :
        print(f"{str(ccd_fpath)} is not exist...")
        doing_status = True
    else : 
        t = os.path.getmtime(ccd_fpath)
        ccd_fpath_dt = datetime.fromtimestamp(t)
        print("ccd_fpath_dt: ", ccd_fpath_dt)
        if ccd_fpath_dt < datetime.now() + timedelta(weeks=checkTime) :
            print(f"{str(ccd_fpath)} is {checkTime} weeks old...")
            doing_status = True
        
    if doing_status == False :
        summary_ccd = pd.read_csv(str(ccd_fpath))
                
    else : 
        DOINGSUBDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
        print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
        print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

        remove = 'Cal'
        DOINGSUBDIRs = [x for x in DOINGSUBDIRs if remove not in x]
        print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
        print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

        summary_suball = pd.DataFrame()
        print(summary_suball)
        
        for DOINGSUBDIR in DOINGSUBDIRs[:] :
            DOINGSUBDIR = Path(DOINGSUBDIR)
            print("DOINGSUBDIR", DOINGSUBDIR)
            
            # save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
            save_fpath2 = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            if save_fpath2.exists():
                os.remove(str(save_fpath2))
                print (f"{str(save_fpath2)} is deleted...")
            
            #save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            save_fpath = DOINGSUBDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            print (f"Starting...\n{DOINGSUBDIR.name}")

            if not save_fpath.exists() :
                print(f"{str(save_fpath)} is not exist...")
                doing_status = True
            else :
                t = os.path.getmtime(save_fpath)
                save_fpath_dt = datetime.fromtimestamp(t)
                #print("save_fpath_dt: ", save_fpath_dt)
                if save_fpath_dt < datetime.now() + timedelta(weeks=checkTime) :
                    print(f"{str(ccd_fpath)} is {checkTime} weeks old...")
                    doing_status = True
            
            if doing_status == False :
                summary_suball = pd.read_csv(str(save_fpath))

            else : 
                fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
                print("fits_in_dir", fits_in_dir)
                print("len(fits_in_dir)", len(fits_in_dir))
                if len(fits_in_dir) == 0 :
                    print(f"There is no fits fils in {DOINGSUBDIR}")
                    summary = pd.DataFrame()
                    pass
                else : 
                    summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
                            output = save_fpath,
                            verbose = True,
                            )
            summary.to_csv(str(save_fpath))
            print(f"{save_fpath} is created...")
            summary_suball = pd.concat([summary_suball, summary], axis = 0)

        summary_suball.reset_index(inplace=True)
        summary_suball.to_csv(str(DOINGDIR.parents[1] / f"summary_{DOINGSUBDIR.parts[-2]}.csv"))
        print(f"{save_fpath} is created...")
    summary_ccd = pd.concat([summary_ccd, summary_suball], axis = 0)
    summary_ccd.to_csv(str(ccd_fpath))
    print(f"{ccd_fpath} is created...")  

ccd_fpath /mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_1bin/QSI683ws_1bin.csv
Empty DataFrame
Columns: []
Index: []
/mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_1bin/QSI683ws_1bin.csv is not exist...


In [ ]:
# #%%
# #######################################################
# for DOINGDIR in DOINGDIRs[:] :
#     DOINGDIR = Path(DOINGDIR)
#     ccd_fpath = Path(f"{DOINGDIR/DOINGDIR.parts[-1]}.csv")
#     print("ccd_fpath", ccd_fpath)

#     summary_ccd = pd.DataFrame()
#     print(summary_ccd)   

#     if ccd_fpath.exists() :
#         print(f"{str(ccd_fpath)} is already exist...")
#         t = os.path.getmtime(ccd_fpath)
#         ccd_fpath_dt = datetime.fromtimestamp(t)
#         print("ccd_fpath_dt: ", ccd_fpath_dt)
#         # if ccd_fpath_dt > datetime.now() + timedelta(days=0) :
#         #     print(f"{str(ccd_fpath)} is not old...")
#         summary_ccd = pd.read_csv(str(ccd_fpath))
                
#     else : 
#         DOINGSUBDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
#         print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
#         print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

#         remove = 'Cal'
#         DOINGSUBDIRs = [x for x in DOINGSUBDIRs if remove not in x]
#         print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
#         print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

#         summary_suball = pd.DataFrame()
#         print(summary_suball)
        
#         for DOINGSUBDIR in DOINGSUBDIRs[:] :
#             DOINGSUBDIR = Path(DOINGSUBDIR)
#             print("DOINGSUBDIR", DOINGSUBDIR)
            
#             # save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
#             save_fpath2 = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
#             if save_fpath2.exists():
#                 os.remove(str(save_fpath2))
#                 print (f"{str(save_fpath2)} is deleted...")
            
#             #save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
#             save_fpath = DOINGSUBDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
#             print (f"Starting...\n{DOINGSUBDIR.name}")

#             if save_fpath.exists() :
#                 print(f"{str(save_fpath)} is already exist...")
#                 t = os.path.getmtime(save_fpath)
#                 save_fpath_dt = datetime.fromtimestamp(t)
#                 #print("save_fpath_dt: ", save_fpath_dt)
#                 #if save_fpath_dt > datetime.now() + timedelta(days=0) :
#                 #print(f"{str(ccd_fpath)} is not old...")
#                 summary = pd.read_csv(str(save_fpath))

#             else : 
#                 fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
#                 print("fits_in_dir", fits_in_dir)
#                 print("len(fits_in_dir)", len(fits_in_dir))
#                 if len(fits_in_dir) == 0 :
#                     print(f"There is no fits fils in {DOINGSUBDIR}")
#                     summary = pd.DataFrame()
#                     pass
#                 else : 
#                     summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
#                             output = save_fpath,
#                             verbose = True,
#                             )
#             summary.to_csv(str(save_fpath))
#             print(f"{save_fpath} is created...")
#             summary_suball = pd.concat([summary_suball, summary], axis = 0)

#         summary_suball.reset_index(inplace=True)
#         summary_suball.to_csv(str(DOINGDIR.parents[1] / f"summary_{DOINGSUBDIR.parts[-2]}.csv"))
#         print(f"{save_fpath} is created...")
#     summary_ccd = pd.concat([summary_ccd, summary_suball], axis = 0)
#     summary_ccd.to_csv(str(ccd_fpath))
#     print(f"{ccd_fpath} is created...")  

ccd_fpath /mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_1bin/QSI683ws_1bin.csv
Empty DataFrame
Columns: []
Index: []


In [ ]:
#%%
#######################################################
checkTime = -2
for DOINGDIR in DOINGDIRs[:] :
    doing_status = False
    DOINGDIR = Path(DOINGDIR)
    ccd_fpath = Path(f"{DOINGDIR/DOINGDIR.parts[-1]}.csv")
    print("ccd_fpath", ccd_fpath)

    summary_ccd = pd.DataFrame()
    print(summary_ccd)   

    if not ccd_fpath.exists() :
        print(f"{str(ccd_fpath)} is not exist...")
        doing_status = True
    else : 
        t = os.path.getmtime(ccd_fpath)
        ccd_fpath_dt = datetime.fromtimestamp(t)
        print("ccd_fpath_dt: ", ccd_fpath_dt)
        if ccd_fpath_dt < datetime.now() + timedelta(weeks=checkTime) :
            print(f"{str(ccd_fpath)} is {checkTime} weeks old...")
            doing_status = True
        
    if doing_status == False :
        summary_ccd = pd.read_csv(str(ccd_fpath))
                
    else : 
        DOINGSUBDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
        print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
        print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

        remove = 'Cal'
        DOINGSUBDIRs = [x for x in DOINGSUBDIRs if remove not in x]
        print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
        print ("len(DOINGSUBDIRs): ", len(DOINGSUBDIRs))

        summary_suball = pd.DataFrame()
        print(summary_suball)
        
        for DOINGSUBDIR in DOINGSUBDIRs[:] :
            DOINGSUBDIR = Path(DOINGSUBDIR)
            print("DOINGSUBDIR", DOINGSUBDIR)
            
            # save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
            save_fpath2 = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            if save_fpath2.exists():
                os.remove(str(save_fpath2))
                print (f"{str(save_fpath2)} is deleted...")
            
            #save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            save_fpath = DOINGSUBDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
            print (f"Starting...\n{DOINGSUBDIR.name}")

            if not save_fpath.exists() :
                print(f"{str(save_fpath)} is not exist...")
                doing_status = True
            else :
                t = os.path.getmtime(save_fpath)
                save_fpath_dt = datetime.fromtimestamp(t)
                #print("save_fpath_dt: ", save_fpath_dt)
                if save_fpath_dt < datetime.now() + timedelta(weeks=checkTime) :
                    print(f"{str(ccd_fpath)} is {checkTime} weeks old...")
                    doing_status = True
            
            if doing_status == False :
                summary = pd.read_csv(str(save_fpath))

            else : 
                fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
                print("fits_in_dir", fits_in_dir)
                print("len(fits_in_dir)", len(fits_in_dir))
                if len(fits_in_dir) == 0 :
                    print(f"There is no fits fils in {DOINGSUBDIR}")
                    summary = pd.DataFrame()
                    pass
                else : 
                    summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
                            output = save_fpath,
                            verbose = True,
                            )
            summary.to_csv(str(save_fpath))
            print(f"{save_fpath} is created...")
            summary_suball = pd.concat([summary_suball, summary], axis = 0)

        summary_suball.reset_index(inplace=True)
        summary_suball.to_csv(str(DOINGDIR.parents[1] / f"summary_{DOINGSUBDIR.parts[-2]}.csv"))
        print(f"{save_fpath} is created...")
    summary_ccd = pd.concat([summary_ccd, summary_suball], axis = 0)
    summary_ccd.to_csv(str(ccd_fpath))
    print(f"{ccd_fpath} is created...")  

NameError: name 'DOINGDIRs' is not defined

In [23]:
DOINGSUBDIR = Path("'/mnt/Rdata/OBS_data/A3_CCD_obs_raw/QSI683ws_1bin/LIGHT_TMB130ss-x75/M27_LIGHT_-_2017-09-13_-_TMB130ss-x75_QSI683ws_-_1bin'")
save_fpath = DOINSUBGDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
print(save_fpath)
print(str(save_fpath.parents[0] / f"summary_{save_fpath.parts[-2]}.csv"))
#save_fpath = DOINSUBGDIR.parents[1] / f"summary_{DOINGSUBDIR.parts[-2]}.csv"


/mnt/Rdata/OBS_data/A2_CCD_newUpdated_files/summary_M27_LIGHT_-_2017-09-13_-_TMB130ss-x75_QSI683ws_-_1bin'.csv
/mnt/Rdata/OBS_data/A2_CCD_newUpdated_files/summary_A2_CCD_newUpdated_files.csv


In [ ]:
# #%%
# for DOINGDIR in DOINGDIRs[:2] :
#     DOINGDIR = Path(DOINGDIR)
#     DOINGSUBDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(str(DOINGDIR)))
#     ccd_fpath = Path(f"{DOINGDIR/DOINGDIR.parts[-1]}.csv")
#     print("ccd_fpath", ccd_fpath)

#     if ccd_fpath.exists() or False:
#         print(f"{str(ccd_fpath)} is already exist...")

#     else : 
#         summary_all = pd.DataFrame()

#         for DOINGSUBDIR in DOINGSUBDIRs[:10] :
#             DOINGSUBDIR = Path(DOINGSUBDIR)
#             print("DOINGSUBDIR", DOINGSUBDIR)
#             fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
#             print("fits_in_dir", fits_in_dir)
#             print("len(fits_in_dir)", len(fits_in_dir))
#             if len(fits_in_dir) == 0 :
#                 print(f"There is no fits fils in {DOINGSUBDIR}")
#                 pass
#             else : 
#                 save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
#                 save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
#                 print (f"Starting...\n{DOINGSUBDIR.name}")
#                 if save_fpath2.exists():
#                     os.remove(str(save_fpath2))
#                     print (f"{str(save_fpath2)} is deleted...")
                
#                 if save_fpath.exists():
#                     print(f"{str(save_fpath)} is already exist...")
#                     summary = pd.read_csv(str(save_fpath))
                
#                 else : 
#                     summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
#                                 output = save_fpath,
#                                 verbose = False
#                                 )
#                     print(f"{save_fpath} is created...")
#                 summary_all = pd.concat([summary_all, summary], axis = 0)
        
#         summary_all.reset_index(inplace=True)
#         summary_all.to_csv(str(ccd_fpath))
#         print(f"{ccd_fpath} is created...")


In [ ]:
checkTime = 1
for CCDDIR in CCDDIRs[:] :
    doing_status = False
    CCDDIR = Path(CCDDIR)
    ccd_fpath = Path(CCDDIR.parents[0] / f"summary_{CCDDIR.parts[-1]}.csv")
    print("ccd_fpath", ccd_fpath)

    summary_ccd = pd.DataFrame(columns=['file'])
    print(summary_ccd)   

    if not ccd_fpath.exists() :
        print(f"{str(ccd_fpath)} is not exist...")
        doing_status = True
    else : 
        t = os.path.getmtime(ccd_fpath)
        ccd_fpath_dt = datetime.fromtimestamp(t)
        print("ccd_fpath_dt: ", ccd_fpath_dt)
        doing_status = False
        if ccd_fpath_dt < datetime.now() + timedelta(days=checkTime) :
            print(f"{str(ccd_fpath)} is older more than {checkTime} days ...")
            doing_status = True
    
    ########################
    #doing_status = True
    ########################

    if doing_status == False :
        summary_suball = pd.read_csv(str(ccd_fpath))
                
    else : 
        CCDOPTICDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(str(CCDDIR)))
        print ("CCDOPTICDIRs: ", CCDOPTICDIRs)
        print ("len(CCDOPTICDIRs): ", len(CCDOPTICDIRs))

        # remove = 'Cal'
        # CCDOPTICDIRs = [x for x in CCDOPTICDIRs if remove not in x]
        # print ("DOINGSUBDIRs: ", DOINGSUBDIRs)
        # print ("len(CCDOPTICDIRs): ", len(CCDOPTICDIRs))
        doing_status = False
        CCDOPTICDIR = Path(CCDOPTICDIR)
        ccdoptic_fpath = Path(CCDOPTICDIR.parents[0] / f"summary_{CCDOPTICDIR.parts[-1]}.csv")
        print("ccdoptic_fpath", ccdoptic_fpath)

        summary_suball = pd.DataFrame(columns=['file'])
        print(summary_suball)

        if not ccdoptic_fpath.exists() :
            print(f"{str(ccdoptic_fpath)} is not exist...")
            doing_status = True
        else : 
            t = os.path.getmtime(ccdoptic_fpath)
            ccd_fpath_dt = datetime.fromtimestamp(t)
            print("ccdoptic_fpath_dt: ", ccdoptic_fpath_dt)
            doing_status = False
            if ccdoptic_fpath_dt < datetime.now() + timedelta(days=checkTime) :
                print(f"{str(ccdoptic_fpath)} is older more than {checkTime} days ...")
                doing_status = True

        if doing_status == False :
        summary_suball = pd.read_csv(str(ccdoptic_fpath))
        
        else : 
            
            for DOINGSUBDIR in CCDOPTICDIRs[:] :
                doing_status = True
                DOINGSUBDIR = Path(DOINGSUBDIR)
                print("DOINGSUBDIR", DOINGSUBDIR)
                
                # # save_fpath2 = DOINGSUBDIR / f"{DOINGSUBDIR.parts[-1]}.csv"
                # save_fpath2 = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
                # save_fpath2 = DOINGSUBDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
                # if save_fpath2.exists():
                #     os.remove(str(save_fpath2))
                #     print (f"{str(save_fpath2)} is deleted...")
                
                #save_fpath = DOINGSUBDIR / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
                save_fpath = DOINGSUBDIR.parents[0] / f"summary_{DOINGSUBDIR.parts[-1]}.csv"
                print (f"Starting...\n{DOINGSUBDIR.name}")

                if not save_fpath.exists() :
                    print(f"{str(save_fpath)} is not exist...")
                    doing_status = True
                else :
                    t = os.path.getmtime(save_fpath)
                    save_fpath_dt = datetime.fromtimestamp(t)
                    #print("save_fpath_dt: ", save_fpath_dt)
                    if save_fpath_dt < datetime.now() + timedelta(days=checkTime) :
                        print(f"{str(ccd_fpath)} is older more then {checkTime} days ...")
                        doing_status = True
                    else:
                        doing_status = False
                
                ########################
                #doing_status = True
                ########################
                if doing_status == False :
                    summary = pd.read_csv(str(save_fpath))

                else : 
                    fits_in_dir = sorted(list(DOINGSUBDIR.glob('*.fit*')))
                    print("fits_in_dir", fits_in_dir)
                    print("len(fits_in_dir)", len(fits_in_dir))
                    if len(fits_in_dir) == 0 :
                        print(f"There is no fits fils in {DOINGSUBDIR}")
                        summary = pd.DataFrame(columns=['file'])
                        pass
                    else : 
                        summary = yfu.make_summary(DOINGSUBDIR/"*.fit*",
                                output = save_fpath,
                                verbose = True,
                                )
                summary.set_index('file')
                summary.to_csv(str(save_fpath))
                print(f"{save_fpath} is created...")
                summary_suball = pd.concat([summary_suball, summary], axis = 0)
        
        summary_suball.set_index('file')
        summary_suball.to_csv(str(ccdoptic_fpath))
        print(f"{ccdoptic_fpath} is created...")
    
    summary_ccd = pd.concat([summary_ccd, summary_suball], axis = 0)
    summary_ccd.set_index('file')
    summary_ccd.to_csv(str(ccd_fpath))
    print(f"{ccd_fpath} is created...(ccd_fpath)")  

In [ ]:
# if ccd_fpath.exists():
#     print(f"{BASEDIR/ccd_dir}.csv is already exist...")
# else : 
#     summary_all = pd.DataFrame()
    

In [ ]:
# summary_all = pd.DataFrame()
# save_fpath2 = fpath/f"{fpath.parts[-1]}.csv"
# save_fpath = fpath/f"summary_{fpath.parts[-1]}.csv"
# print (f"Starting...\n{fpath.name}")
# if save_fpath2.exists():
#     os.remove(str(save_fpath2))
#     print (f"{str(save_fpath2)} is deleted...")

# if save_fpath.exists():
#     print(f"{str(save_fpath)} is already exist...")
#     summary = pd.read_csv(str(save_fpath))

# else : 
#     summary = yfu.make_summary(fpath/"*.fit*",
#                 output = save_fpath,
#                 verbose = False
#                 )
#     print(f"{save_fpath} is created...")
# summary_all = pd.concat([summary_all, summary], axis = 0)
# summary_all.to_csv(f"{BASEDIR/ccd_dir}.csv")
# summary_all.reset_index(inplace=True)
# print(f"{BASEDIR/ccd_dir}.csv is created...")